In [ ]:
%pip install -r requirements.txt
%pip install pandas
%pip install kaleido
%pip install plotly

In [4]:
import numpy as np
import pandas as pd
import torch
from scipy.stats import norm

test_df = pd.read_csv('dataset/processed/test.csv')  
train_df = pd.read_csv('dataset/processed/training.csv')  
validation_df = pd.read_csv('dataset/processed/validation.csv') 

split = np.array_split(validation_df, 2)

validation_df_error_means = split[0]
validation_df_classification = split[1]

import plotly.graph_objects as go

torch.set_default_dtype(torch.float64)


In [ ]:
trace1 = go.Scatter(
    x = train_df.timestamp,
    y = train_df.value,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = 'Signal Value Plot',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1], layout=layout)
fig.show()


In [5]:
from sklearn.preprocessing import StandardScaler


class AnomalyDetectionDataset(torch.utils.data.Dataset):

    def __init__(self, data, n_past=14, n_future=1):
        self.n_future = n_future
        self.n_past = n_past
        self.x = []
        self.actual = []

        self.scaler = StandardScaler()

        data_columns = ['value', 'predicted', "is_anomaly"]

        x = data[data_columns].astype(float)
        self.scaler = self.scaler.fit(x)
        x_scaled = self.scaler.transform(x)

        self.X = []
        self.Y = []
        self.IsAnomaly = []

        for i in range(self.n_past, len(x_scaled) - self.n_future + 1):
            self.X.append(  torch.tensor(x_scaled[i - self.n_past:i, 0: 2])  )
            self.Y.append(  torch.tensor(x_scaled[i: i + self.n_future, 0: 2 ])  )
            self.IsAnomaly.append(  torch.tensor(x_scaled[i:i + self.n_future, 2])  )

        assert len(self.X) == len(self.Y)

    def __len__(self):
        return len(self.X)


    def __getitem__(self, idx):
        return {
            "x": self.X[idx],
            "y": self.Y[idx],
            "is_anomaly": self.IsAnomaly[idx]
        }


def collate_batch(batch):
    x = []
    y = []
    is_anomaly = []

    for item in batch:
        x.append(item["x"])
        y.append(item["y"])
        is_anomaly.append(item["is_anomaly"])

    x = torch.nn.utils.rnn.pad_sequence(x, batch_first=True)
    y = torch.nn.utils.rnn.pad_sequence(y, batch_first=True)
    is_anomaly = torch.nn.utils.rnn.pad_sequence(is_anomaly, batch_first=True)

    return {
        "x": x,
        "y": y,
        "is_anomaly": is_anomaly
    }

In [57]:
import torch.nn as nn
import torch


class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class EncoderDecoderConvLSTM(nn.Module):
    def __init__(self, nf, in_chan,height):
        super(EncoderDecoderConvLSTM, self).__init__()

        """ ARCHITECTURE 

        # Encoder (ConvLSTM)
        # Encoder Vector (final hidden state of encoder)
        # Decoder (ConvLSTM) - takes Encoder Vector as input
        # Decoder (3D CNN) - produces regression predictions for our model

        """
        self.height = height
        self.encoder_1_convlstm = ConvLSTMCell(input_dim=in_chan,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.encoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_1_convlstm = ConvLSTMCell(input_dim=nf,  # nf + 1
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_CNN = nn.Conv3d(in_channels=nf,
                                     out_channels=1,
                                     kernel_size=(1, self.height+2, 3),
                                     padding=(0, 1, 1))
        # self.fc1 = nn.Linear(14 * 3 * batch_len, 24)
        # self.fc2 = nn.Linear(24, 12)
        # self.fc3 = nn.Linear(12, batch_len)


    def autoencoder(self, x, seq_len, future_step, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4):

        outputs = []

        # encoder
        for t in range(seq_len):
            h_t, c_t = self.encoder_1_convlstm(input_tensor=x[:, t, :, :],
                                               cur_state=[h_t, c_t])  # we could concat to provide skip conn here
            h_t2, c_t2 = self.encoder_2_convlstm(input_tensor=h_t,
                                                 cur_state=[h_t2, c_t2])  # we could concat to provide skip conn here

        # encoder_vector
        encoder_vector = h_t2

        # decoder
        for t in range(future_step):
            h_t3, c_t3 = self.decoder_1_convlstm(input_tensor=encoder_vector,
                                                 cur_state=[h_t3, c_t3])  # we could concat to provide skip conn here
            h_t4, c_t4 = self.decoder_2_convlstm(input_tensor=h_t3,
                                                 cur_state=[h_t4, c_t4])  # we could concat to provide skip conn here
            encoder_vector = h_t4
            outputs += [h_t4]  # predictions

        outputs = torch.stack(outputs, 1) # torch.Size([1, 1, nf, 14, 2])
        # print(outputs.shape)
        outputs = outputs.permute(0, 2, 1, 3, 4) #torch.Size([1, nf, 1, 14, 2])
        # print(outputs.shape)
        outputs = self.decoder_CNN(outputs) # torch.Size([1, 1, 1, nf, 2])
        # outputs = torch.flatten(outputs) # torch.Size([1, 1, 1, nf, 2])
        outputs = torch.nn.Sigmoid()(outputs)
        return outputs

    def forward(self, x, future_seq=0):

        """
        Parameters
        ----------
        input_tensor:
            5-D Tensor of shape (b, t, c, h, w)        #   batch, time, channel, height, width
        """

        # find size of different input dimensions
        b, seq_len, _, h, w = x.size()

        # initialize hidden states
        h_t, c_t = self.encoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t2, c_t2 = self.encoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t3, c_t3 = self.decoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t4, c_t4 = self.decoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))

        # autoencoder forward
        outputs = self.autoencoder(x, seq_len, future_seq, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4)

        return outputs

In [ ]:
from itertools import cycle
from tqdm import trange



batch_len = 100
input_channels = 1 # 1
output_channels = 1 # 1
height = 14 ## Number of previous datapoints to use
width = 2 ## Number of features per datapoints


losses = {
    "train":[],
    "val":[]
}

train_dataset = AnomalyDetectionDataset(train_df, n_past=height, n_future=output_channels)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_len, shuffle=False, collate_fn=collate_batch
)

# use the first half of the normal dev data as the dev set for the LSTM
split = np.array_split(validation_df, 2)
val_data = split[0]
val_dataset = AnomalyDetectionDataset(val_data, n_past=height,n_future=output_channels)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_len, shuffle=False, collate_fn=collate_batch
)

val_data2 = split[1]
val_dataset2 = AnomalyDetectionDataset(val_data2, n_past=height,n_future=output_channels)
val_dataloader2 = torch.utils.data.DataLoader(
    val_dataset2, batch_size=batch_len, shuffle=False, collate_fn=collate_batch
)

epochs = 100

loss_function = torch.nn.MSELoss(reduction='sum')

len_train = len(train_dataloader)
len_val = len(val_dataloader)


# instantiate model
autoEncoder = EncoderDecoderConvLSTM(nf=64, in_chan=1, batch_len=batch_len)
optimizer = torch.optim.Adam(autoEncoder.parameters(), lr=0.001)

train_iter = cycle(train_dataloader)
val_iter = cycle(val_dataloader)

for epoch in trange(epochs):
    print("epoch", epoch)
    autoEncoder.train()
    plot_loss = []
    plot_val_loss = []

    
    for itr in range(len_train):

        batch = next(train_iter, None)
        loss = 0
        input = torch.tile(batch['x'],(2,1,1))[:batch_len,:,:]
        input = torch.unsqueeze(input, 1)
        input = torch.unsqueeze(input, 1)
        
        pred = autoEncoder(input, output_channels).squeeze()
        y = torch.tile(batch["y"],(2,1,1))[:batch_len,:].squeeze()
        loss = loss_function(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss =loss.item()/batch_len
        plot_loss.append(train_loss)
    print('epoch', epoch, 'train loss', sum(plot_loss)/len_train)
    losses["train"].append(plot_loss)
    autoEncoder.eval()
    for itr in range(len_val):
        
        batch = next(val_iter, None)
        loss=0
        input = torch.tile(batch['x'],(2,1,1))[:batch_len,:,:]
        input = torch.unsqueeze(input, 1)
        input = torch.unsqueeze(input, 1)

        pred = autoEncoder(input, output_channels).squeeze()

        y = torch.tile(batch["y"],(2,1,1))[:batch_len,:].squeeze()

        loss = loss_function(pred, y)
        val_loss = loss.item()/batch_len
        plot_val_loss.append(val_loss)
    print('epoch', epoch, 'val loss', sum(plot_val_loss)/len_val)
    losses["val"].append(plot_val_loss)
    
    

In [ ]:
from functools import reduce

def mean(l):
    return sum(l)/len(l)

train_losses = list(map(mean, losses["train"]))
val_losses = list(map(mean, losses["val"]))


In [ ]:

epochs = [i for i in range(100)]

trace1 = go.Scatter(
    x = epochs,
    y = train_losses,
    mode='lines',
    name = 'train losses'
)
trace2 = go.Scatter(
    x = epochs,
    y = val_losses,
    mode='lines',
    name = 'val losses'
)

layout = go.Layout(
    title = 'Trian vs Validation loss',
    xaxis = {'title' : "epoch"},
    yaxis = {'title' : "loss"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()


In [ ]:
torch.save(autoEncoder.state_dict(), "models/convlstmae/100batch_100epoch_64hidden_14prev_1future_0_001lr_adam/model.pt")

In [ ]:
# reload the best model
model = EncoderDecoderConvLSTM(nf=64, in_chan=1, batch_len=batch_len)
model.load_state_dict(torch.load("models/convlstmae/100batch_100epoch_64hidden_14prev_1future_0_001lr_adam/model.pt"))


In [35]:
def evaluate(model, dataloader, loss_function,params):
    """
    Returns error means, standard deviations, and loss
    """
    total_loss = 0
    model.eval()

    errors = torch.tensor([])
    batch_len = params["batch_len"]
    output_channels = params["output_channels"]

    with torch.no_grad():
        for batch in dataloader:
            loss = 0
            input = torch.tile(batch['x'],(2,1,1))[:batch_len,:,:]
            input = torch.unsqueeze(input, 1)
            input = torch.unsqueeze(input, 1)

            pred = model(input, output_channels).squeeze()
            y = torch.tile(batch["y"],(2,1,1))[:batch_len,:].squeeze()
            loss = loss_function(pred, y)
            batch_errors = y - pred
            errors = torch.cat([errors, batch_errors])

            total_loss += loss.item()/batch_len

    avg_loss = total_loss / len(dataloader)
    print("Loss:", avg_loss)

    errors_np = errors.numpy()
    means = np.mean(errors_np, axis=0)
    stds = np.std(errors_np, axis=0)
    print("Error means:", means)
    print("Error standard deviations:", stds)

    return errors_np, means, stds, avg_loss

In [43]:
def calculateF1Score(log_likelihoods, value_threshold, is_anomaly, params):
    """
    Precondition: log_likelihoods and is_anomaly should be sorted by predicted
    """
    pred_is_anomaly = log_likelihoods[:,0] < value_threshold

    is_anomaly = is_anomaly > 0
    tp = sum(pred_is_anomaly & is_anomaly)
    fp = sum(pred_is_anomaly & np.logical_not(is_anomaly))
    fn = sum(np.logical_not(pred_is_anomaly) & is_anomaly)

    best_f1 = tp / (tp + 1 / 2 * (fp + fn))
    best_predicted_threshold = log_likelihoods[0,1]

    for i in range(1, len(is_anomaly)):
        if pred_is_anomaly[i - 1]:
            if is_anomaly[i - 1]:
                tp -= 1
                fn += 1
            else:
                fp -= 1

            f1 = tp / (tp + 1 / 2 * (fp + fn))

            if f1 > best_f1:
                best_f1 = f1
                best_predicted_threshold = log_likelihoods[i,1]

    return best_predicted_threshold, best_f1


def calculateF1ScoreFromPredictedThreshold(
    log_likelihoods, value_threshold, predicted_threshold, is_anomaly
):
    pred_is_anomaly = (
        (log_likelihoods[:,0] < value_threshold)
        & (log_likelihoods[:,1] >= predicted_threshold)
    )

    is_anomaly = is_anomaly > 0

    tp = sum(pred_is_anomaly & is_anomaly)
    fp = sum(pred_is_anomaly & np.logical_not(is_anomaly))
    fn = sum(np.logical_not(pred_is_anomaly) & is_anomaly)

    return tp / (tp + 1 / 2 * (fp + fn))


def calculateLogLikelihoods(model, dataloader, loss_function,params):
    errors_np, means, stds, avg_loss = evaluate(model, dataloader, loss_function,params)
    return np.concatenate(([[float('-inf'),float('-inf')]], norm.logpdf(errors_np, loc=means, scale=stds))), errors_np, means, stds, avg_loss


def determineClassificationThresholds(model, dataloader, loss_function,params):
    """
    Returns the best f1, value threshold, and predicted threshold
    """
    best_f1 = -1
    batch_len = params["batch_len"]
    log_likelihoods, errors_np, means, stds, avg_loss = calculateLogLikelihoods(model, dataloader, loss_function,params)

    indices = np.argsort(log_likelihoods, axis=0)[:,1]

    is_anomaly = [float("-inf")]

    for batch in dataloader:
        anomalies = torch.tile(batch['is_anomaly'],(2,1))[:batch_len,:]
        is_anomaly += anomalies[:,0]
    is_anomaly = np.array(is_anomaly)
    
    # sort log likelihoods and anomalies by predicted
    log_likelihoods_sorted = log_likelihoods[indices]
    is_anomaly_sorted = is_anomaly[indices]

    for value_threshold in log_likelihoods_sorted[:,0]:
        predicted_threshold, f1 = calculateF1Score(
            log_likelihoods_sorted, value_threshold, is_anomaly_sorted, params
        )

        if f1 > best_f1:
            best_f1 = f1
            best_value_threshold = value_threshold
            best_predicted_threshold = predicted_threshold

    print("Best f1:", best_f1)
    print("Best value threshold:", best_value_threshold)
    print("Best predicted threshold:", best_predicted_threshold)

    f1 = calculateF1ScoreFromPredictedThreshold(
        log_likelihoods,
        best_value_threshold,
        best_predicted_threshold,
        is_anomaly,
    )
    print("Sanity check with method using predicted threshold:", f1)

    return best_f1, best_value_threshold, best_predicted_threshold


In [62]:
from tqdm import trange

def mean(l):
    return sum(l)/len(l)

def train(epochs, model,optimizer,loss_function, len_train, len_val, train_iter, val_iter, params):


    losses = {
        "train":[],
        "val":[]
    }

    batch_len = params["batch_len"]
    output_channels = params["output_channels"]
    for epoch in range(epochs):
        print("epoch", epoch)
        model.train()
        plot_loss = []
        plot_val_loss = []
        
        
        for itr in trange(len_train):

            batch = next(train_iter, None)
            loss = 0
            input = torch.tile(batch['x'],(batch_len,1,1))[:batch_len,:,:]
            input = torch.unsqueeze(input, 1)
            input = torch.unsqueeze(input, 1)
            
            pred = model(input, output_channels).squeeze()
            y = torch.tile(batch["y"],(batch_len,1,1))[:batch_len,:].squeeze()
            # print(input.shape)
            # print(pred.shape)
            # print(y.shape)
            loss = loss_function(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss =loss.item()/batch_len
            plot_loss.append(train_loss)
        print('epoch', epoch, 'train loss', sum(plot_loss)/len_train)
        losses["train"].append(plot_loss)

        model.eval()
        for itr in trange(len_val):
            
            batch = next(val_iter, None)
            loss=0
            input = torch.tile(batch['x'],(batch_len,1,1))[:batch_len,:,:]
            input = torch.unsqueeze(input, 1)
            input = torch.unsqueeze(input, 1)

            pred = model(input, params["output_channels"]).squeeze()

            y = torch.tile(batch["y"],(batch_len,1,1))[:batch_len,:].squeeze()

            loss = loss_function(pred, y)
            val_loss = loss.item()/batch_len
            plot_val_loss.append(val_loss)

        print('epoch', epoch, 'val loss', sum(plot_val_loss)/len_val)
        losses["val"].append(plot_val_loss)


    model_directory = "models/convlstmae/{}batchlen_{}epoch_{}hidden_{}prev_{}future_{}lr_adam" \
    .format(param["batch_len"], param["epochs"], param["nf"], param["height"], param["output_channels"], param["lr"])

    model_file_name = model_directory + "_model.pt"
    model_train_eval = model_directory + "_train_eval_plot.jpeg"
    model_losses = model_directory + "_loses"

    torch.save(model.state_dict(), model_file_name)


    train_losses = list(map(mean, losses["train"]))
    val_losses = list(map(mean, losses["val"]))


    epochs = [i for i in range(epochs)]

    trace1 = go.Scatter(
        x = epochs,
        y = train_losses,
        mode='lines',
        name = 'train losses'
    )
    trace2 = go.Scatter(
        x = epochs,
        y = val_losses,
        mode='lines',
        name = 'val losses'
    )

    layout = go.Layout(
        title = 'Trian vs Validation loss',
        xaxis = {'title' : "epoch"},
        yaxis = {'title' : "loss"}
    )
    fig = go.Figure(data=[trace1, trace2], layout=layout)

    fig.write_image(model_train_eval)


    return losses

In [59]:
from itertools import cycle

def grid(params,results):
    batch_len = params["batch_len"]
    epochs = params["epochs"]
    height = params["height"] ## Number of previous datapoints to use
    lr = params["lr"]
    nf = params["nf"]
    input_channels = 1 # 1
    output_channels = 1 # 1
    width = 2 ## Number of features per datapoints
    

    train_dataset = AnomalyDetectionDataset(train_df, n_past=height, n_future=output_channels)    
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_len, shuffle=False, collate_fn=collate_batch)

    val_dataset_error_means = AnomalyDetectionDataset(validation_df_error_means, n_past=height,n_future=output_channels)
    val_dataloader_error_means = torch.utils.data.DataLoader(
        val_dataset_error_means, batch_size=batch_len, shuffle=False, collate_fn=collate_batch)

    val_dataset_classification = AnomalyDetectionDataset(validation_df_classification, n_past=height,n_future=output_channels)
    val_dataloader_classification = torch.utils.data.DataLoader(
        val_dataset_classification, batch_size=batch_len, shuffle=False, collate_fn=collate_batch)

    loss_function = torch.nn.MSELoss(reduction='sum')

    len_train = len(train_dataloader)
    len_val = len(val_dataloader_error_means)

    autoEncoder = EncoderDecoderConvLSTM(nf=nf, in_chan=input_channels,height=height)
    optimizer = torch.optim.Adam(autoEncoder.parameters(), lr=lr)

    train_iter = cycle(train_dataloader)
    val_iter = cycle(val_dataloader_error_means)

    train(epochs, autoEncoder, optimizer, loss_function,  len_train, len_val, train_iter, val_iter, params)

    f1, value_threshold, predicted_threshold = determineClassificationThresholds(autoEncoder, val_dataloader_classification, loss_function,params)
    
    if f1 > results["best_f1_grid_search"] :
        results["best_f1_grid_search"] = float("-inf")
        results["best_model_grid_search"] = autoEncoder
        results["best_params_grid_search"] = params
        results["best_value_threshold_grid_search"] = value_threshold
        results["predicted_threshold_grid_search"] = predicted_threshold


In [ ]:
results = {
    "best_f1_grid_search" : float("-inf"),
    "best_model_grid_search" : None,
    "best_params_grid_search" : None,
    "best_value_threshold_grid_search" : None,
    "predicted_threshold_grid_search" : None

}
params = {
    "height": list(reversed([14,21,28,35])),
    "nf": [8,32,64,128,256],
    "epochs": [20],
    # "epochs": [1,10,20,30,40,50],
    "output_channels": [1],
    "lr":[0.001,0.0005,0.0001,0.01,0.005],
    "batch_len": list(reversed([1,20,25,30,35]))
}

best_params = {
        "lr":params["lr"][0],
        "batch_len":params["batch_len"][0],
        "nf": params["nf"][0],
        "height": params["height"][0],
        "epochs": params["epochs"][0],
        "output_channels": params["output_channels"][0]
}

for hyperparameter in params:
    param = best_params.copy()
    for hyperparameter_value in params[hyperparameter]:
        prev_f1 = results["best_f1_grid_search"]
        param[hyperparameter] = hyperparameter_value
        print("params:", param)
        losses = grid(param,results)
        if results["best_f1_grid_search"] > prev_f1:
            best_params[hyperparameter] = hyperparameter_value
        print("Best Params so far: ", best_params)
    

